DATA PREPROCESSING

In [10]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

Load data

In [11]:
train_data = pd.read_csv("Dataset/train.csv", delimiter=",")
test_data = pd.read_csv("Dataset/test.csv", delimiter=",")
data = pd.concat([train_data, test_data])
data.reset_index(inplace=True)
data[:]
data = data.dropna(subset=['Fare', 'Embarked'])
print(data.shape)


(1306, 13)


Data Preprocessing 

1. Name feature

In [12]:
titles = set()
for name in data['Name']:
    titles.add(name.split(',')[1].split('.')[0].strip())

Title_Dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Sir" : "Royalty",
    "Dr": "Officer",
    "Rev": "Officer",
    "the Countess":"Royalty",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Royalty"}

# Edatatract the title from each name
data['Title'] = data['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())
data['Title'] = data.Title.map(Title_Dictionary)

In [13]:
le_title = preprocessing.LabelEncoder()
le_title.fit(['Officer', 'Royalty', 'Mrs', 'Mr', 'Master', 'Miss'])
data['Title'] = le_title.transform(data['Title'])

2. Age feature

In [14]:
grouped_train = data.iloc[:891].groupby(['Sex', 'Pclass', 'Title'])
# Median
grouped_train_median = grouped_train.median('Age')
grouped_train_median = grouped_train_median.reset_index()[['Sex', 'Pclass', 'Title', 'Age']]

def fill_age(row):
    condition = (
        (grouped_train_median['Sex'] == row['Sex']) & 
        (grouped_train_median['Title'] == row['Title']) & 
        (grouped_train_median['Pclass'] == row['Pclass'])
    ) 
    return grouped_train_median[condition]['Age'].values[0]

data['Age'] = data.apply(lambda row: fill_age(row) if np.isnan(row['Age']) else row['Age'], axis = 1)


3. Fare feature

4. Embraked feature

In [15]:
# Initialize and fit LabelEncoder
le_embarked = preprocessing.LabelEncoder()
le_embarked.fit(['Q', 'S', 'C'])

# Transform 'Embarked' column
data['Embarked'] = le_embarked.transform(data['Embarked'])

5. Cabin feature

In [16]:
data.fillna('U', inplace=True)
data['Cabin'] = data['Cabin'].map(lambda c: c[0])
cabin_values = data['Cabin'].unique()
le_cabin = preprocessing.LabelEncoder()
le_cabin.fit(cabin_values)
data['Cabin'] = le_cabin.transform(data['Cabin'])


6. Sex feature

In [17]:
data['Sex'] = data['Sex'].map({'male':1, 'female':0})